In [1]:
%load_ext rich
%load_ext autoreload
%autoreload 2

In [2]:
import polars as pl
import pointblank as pb
import odyssey.core as od

# Set up config
from config import RAW_DATA, PROCESSED_DATA

c:\Users\00113294\Documents\WIP\Validating IPAQ\.venv\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


# THIS IS A PROTOTYPE!! 

It's designed to be thrown away.

The purpose of this activity is to play around with Pointblank for validating the IPAQ data, and identifying discrepancies.

Things to validate include:
- VIG_MINS is equal to VIG_HPD * 60 + VIG_MPD (existing cases - 10170)
    - Q: I've seen multiple cases with VIG_HPD > 3, and VIG_MINS = 180 -> is there a hard cut-off at 180?
    - A: Indeed, according to the following, bouts of activity >3hrs are truncated (see [here](https://ugc.futurelearn.com/uploads/files/bc/c5/bcc53b14-ec1e-4d90-88e3-1568682f32ae/IPAQ_PDF.pdf))
- VIG_MINS is None (missing) or between 0 and 180 (same for MOD and WALK)
- VIG_MET is equal to VIG_MINS * VIG_D * 8 (adjust the factor for MOD (4) and WALK (3.3))
- TOT_MET is equal to VIG_MET + MOD_MET + WALK_MET (existing cases - 10030)
    - Edge case: one or more of these variables is None -> should be converted to 0

In [ ]:
g220 = od.Dataset("G220_Q.sav", PROCESSED_DATA)
lf, meta = g220.load_data()

In [ ]:
lf.collect()

ID,G220_DNWN,G220_MQ_DNWN,G220_THINK_DNWN,G220_THINK_AGE,G220_DWEL_1,G220_LIVE,G220_LIV1,G220_LIV2,G220_LIV3,G220_LIV4,G220_LIV5,G220_LIV8,G220_LIV9,G220_CH,G220_PCSX1,G220_PCSX2,G220_PCSX3,G220_PCSX4,G220_PCSX5,G220_PCBY1,G220_PCBY2,G220_PCBY3,G220_PCBY4,G220_PCBY5,G220_ED33,G220_ED34,G220_ED35,G220_ED36_1,G220_OCC1,G220_OCC2,G220_OCC3,G220_OCC4,G220_OCC5,G220_OCC6,G220_OCC7,G220_OCC8,…,G220_THINK_47,G220_THINK_48,G220_THINK_49,G220_THINK_50,G220_AUTISM,G220_AUTISM_YR,G220_ALC_MAX,G220_ALC_PW_MEAN,G220_ALC_PW_TOT,G220_DASS2,G220_DASS3,G220_DASS4,G220_DASS6,G220_DASS8,G220_DASS10,G220_DASS12,G220_DASS18,G220_DASS20,G220_DASS22,G220_DASS25,G220_DASS26,G220_DASS28,G220_DASS31,G220_DASS32,G220_DASS34,G220_DASS38,G220_DASS39,G220_DASS40,G220_DASS41,G220_DASS42,G220_DASS_DEP_SCORE,G220_DASS_DEP_CAT,G220_DASS_ANX_SCORE,G220_DASS_ANX_CAT,G220_DASS_STR_SCORE,G220_DASS_STR_CAT,G220_DASS_TOT_SCORE
f64,date,date,date,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
10010.0,2010-03-11,2010-03-11,null,null,2.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,8.0,8.0,8.0,8.0,8.0,null,null,null,null,null,2.0,1.0,1.0,2.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,…,null,null,null,null,0.0,888.0,12.0,14.0,2.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,12.0,2.0,6.0,1.0,12.0,1.0,30.0
10020.0,2010-05-01,2010-05-16,2010-02-27,20.34,4.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,8.0,8.0,8.0,8.0,null,null,null,null,null,111.0,1.0,0.0,888.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,4.0,4.0,3.0,4.0,0.0,888.0,5.0,8.0,1.14,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,4.0,1.0,4.0,1.0,10.0
10030.0,2011-04-04,2011-04-04,null,null,2.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,8.0,8.0,8.0,8.0,8.0,null,null,null,null,null,2.0,2.0,0.0,888.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,…,null,null,null,null,0.0,888.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,2.0,1.0,-99.0,0.0,0.0,1.0,2.0,0.0,1.0,8.0,1.0,8.0,2.0,14.0,1.0,30.0
10040.0,2010-05-12,2010-04-12,2010-03-01,20.29,2.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,8.0,8.0,8.0,8.0,8.0,null,null,null,null,null,2.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,…,3.0,4.0,2.0,4.0,0.0,888.0,4.0,4.0,0.57,0.0,1.0,0.0,0.0,1.0,1.0,0.0,2.0,1.0,1.0,0.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,0.0,0.0,2.0,16.0,3.0,4.0,1.0,16.0,2.0,36.0
10050.0,2010-03-09,2010-03-13,2010-03-02,20.35,2.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,8.0,8.0,8.0,8.0,8.0,null,null,null,null,null,1.0,2.0,1.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,…,3.0,3.0,4.0,3.0,0.0,888.0,15.0,39.0,5.57,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,8.0,1.0,4.0,1.0,2.0,1.0,14.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
54540.0,null,null,2011-12-11,19.68,null,999.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,null,null,null,null,null,null,null,null,null,null,null,999.0,null,null,999.0,null,null,null,null,null,null,null,null,…,3.0,3.0,4.0,3.0,0.0,888.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
54550.0,2011-10-22,2011-10-23,2011-10-20,19.59,2.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,8.0,8.0,8.0,8.0,8.0,null,null,null,null,null,111.0,1.0,0.0,888.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,…,3.0,2.0,2.0,3.0,0.0,888.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,2.0,1.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,1.0,14.0,3.0,12.0,1.0,26.0
54580.0,2011-09-29,2011-09-29,2011-09-29,19.47,2.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,8.0,8.0,8.0,8.0,8.0,null,null,null,null,null,2.0,1.0,1.0,3.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,…,3.0,3.0,2.0,3.0,0.0,888.0,4.0,10.0,1.43,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [ ]:
validation = (
    pb.Validate(
        data=pb.load_dataset(dataset="game_revenue", tbl_type="polars")
    )
    .col_vals_ge(
        columns=pb.matches("rev|dur"),  # check values in columns having 'rev' or 'dur' in name
        value=0
    )
    .col_vals_regex(
        columns=pb.ends_with("_id"),    # check values in columns with names ending in '_id'
        pattern=r"^[A-Z]{12}\d{3}"
    )
    .col_vals_not_null(
        columns=pb.last_n(2)            # check that the last two columns don't have Null values
    )
    .interrogate()
)

validation

Validate(
    data=shape: (2_000, 11)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ player_id ┆ session_i ┆ session_s ┆ time      ┆ … ┆ session_d ┆ start_day ┆ acquisiti ┆ country  │
│ ---       ┆ d         ┆ tart      ┆ ---       ┆   ┆ uration   ┆ ---       ┆ on        ┆ ---      │
│ str       ┆ ---       ┆ ---       ┆ datetime[ ┆   ┆ ---       ┆ date      ┆ ---       ┆ str      │
│           ┆ str       ┆ datetime[ ┆ μs, UTC]  ┆   ┆ f64       ┆           ┆ str       ┆          │
│           ┆           ┆ μs, UTC]  ┆           ┆   ┆           ┆           ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ ECPANOIXL ┆ ECPANOIXL ┆ 2015-01-0 ┆ 2015-01-0 ┆ … ┆ 16.3      ┆ 2015-01-0 ┆ google    ┆ Germany  │
│ ZHF896    ┆ ZHF896-eo ┆ 1         ┆ 1         ┆   ┆           ┆ 1         ┆           ┆          │
│           ┆ l2j8bs    ┆ 01:31:03  ┆ 01:31:27  ┆   ┆           ┆           ┆           ┆          │
│           ┆           ┆ UTC       ┆ UTC       ┆   ┆           ┆           ┆           ┆          │
│ ECPANOIXL ┆ ECPANOIXL ┆ 2015-01-0 ┆ 2015-01-0 ┆ … ┆ 16.3      ┆ 2015-01-0 ┆ google    ┆ Germany  │
│ ZHF896    ┆ ZHF896-eo ┆ 1         ┆ 1         ┆   ┆           ┆ 1         ┆           ┆          │
│           ┆ l2j8bs    ┆ 01:31:03  ┆ 01:36:57  ┆   ┆           ┆           ┆           ┆          │
│           ┆           ┆ UTC       ┆ UTC       ┆   ┆           ┆           ┆           ┆          │
│ ECPANOIXL ┆ ECPANOIXL ┆ 2015-01-0 ┆ 2015-01-0 ┆ … ┆ 16.3      ┆ 2015-01-0 ┆ google    ┆ Germany  │
│ ZHF896    ┆ ZHF896-eo ┆ 1         ┆ 1         ┆   ┆           ┆ 1         ┆           ┆          │
│           ┆ l2j8bs    ┆ 01:31:03  ┆ 01:37:45  ┆   ┆           ┆           ┆           ┆          │
│           ┆           ┆ UTC       ┆ UTC       ┆   ┆           ┆           ┆           ┆          │
│ ECPANOIXL ┆ ECPANOIXL ┆ 2015-01-0 ┆ 2015-01-0 ┆ … ┆ 16.3      ┆ 2015-01-0 ┆ google    ┆ Germany  │
│ ZHF896    ┆ ZHF896-eo ┆ 1         ┆ 1         ┆   ┆           ┆ 1         ┆           ┆          │
│           ┆ l2j8bs    ┆ 01:31:03  ┆ 01:42:33  ┆   ┆           ┆           ┆           ┆          │
│           ┆           ┆ UTC       ┆ UTC       ┆   ┆           ┆           ┆           ┆          │
│ ECPANOIXL ┆ ECPANOIXL ┆ 2015-01-0 ┆ 2015-01-0 ┆ … ┆ 35.2      ┆ 2015-01-0 ┆ google    ┆ Germany  │
│ ZHF896    ┆ ZHF896-hd ┆ 1         ┆ 1         ┆   ┆           ┆ 1         ┆           ┆          │
│           ┆ u9jkls    ┆ 11:50:02  ┆ 11:55:20  ┆   ┆           ┆           ┆           ┆          │
│           ┆           ┆ UTC       ┆ UTC       ┆   ┆           ┆           ┆           ┆          │
│ …         ┆ …         ┆ …         ┆ …         ┆ … ┆ …         ┆ …         ┆ …         ┆ …        │
│ NAOJRDMCS ┆ NAOJRDMCS ┆ 2015-01-2 ┆ 2015-01-2 ┆ … ┆ 25.8      ┆ 2015-01-1 ┆ organic   ┆ Norway   │
│ EBI281    ┆ EBI281-j2 ┆ 1         ┆ 1         ┆   ┆           ┆ 1         ┆           ┆          │
│           ┆ vs9ilp    ┆ 01:57:50  ┆ 02:02:50  ┆   ┆           ┆           ┆           ┆          │
│           ┆           ┆ UTC       ┆ UTC       ┆   ┆           ┆           ┆           ┆          │
│ NAOJRDMCS ┆ NAOJRDMCS ┆ 2015-01-2 ┆ 2015-01-2 ┆ … ┆ 25.8      ┆ 2015-01-1 ┆ organic   ┆ Norway   │
│ EBI281    ┆ EBI281-j2 ┆ 1         ┆ 1         ┆   ┆           ┆ 1         ┆           ┆          │
│           ┆ vs9ilp    ┆ 01:57:50  ┆ 02:22:14  ┆   ┆           ┆           ┆           ┆          │
│           ┆           ┆ UTC       ┆ UTC       ┆   ┆           ┆           ┆           ┆          │
│ RMOSWHJGE ┆ RMOSWHJGE ┆ 2015-01-2 ┆ 2015-01-2 ┆ … ┆ 8.4       ┆ 2015-01-1 ┆ other_cam ┆ France   │
│ LCI675    ┆ LCI675-vb ┆ 1         ┆ 1         ┆   ┆           ┆ 0         ┆ paign     ┆          │
│           ┆ hcsmtr    ┆ 02:39:48  ┆ 02:40:00  ┆   ┆           ┆           ┆           ┆          │
│           ┆          